## Importing

In [1]:
import fiona
from pathlib import Path
import pandas as pd
from osgeo import ogr, gdal, gdalconst, osr
from osgeo_utils import gdal_merge
import numpy as np


## Paths

In [2]:
tiff_path = Path(r'D:\Ferrari\Projects\deforestation-forescast\exp-1km\tiff')
shp_path = Path(r'D:\Ferrari\Projects\deforestation-forescast\exp-1km\shp')

for f in shp_path.glob('*.shp'):
    print(f.name)

predictions-ibama.shp
predictions-resunet.shp
reference-ibama.shp
reference-resunet.shp


## Conversion

In [3]:
cells_path = shp_path / 'predictions-resunet.shp'
output_path = tiff_path / 'predictions-resunet.tif'
col = fiona.open(cells_path)
attr_list = [att for att in col.schema['properties'].keys() if str(att).startswith('b')]
sorted_attr_dict = sorted(attr_list)

v_cells_data = ogr.Open(str(cells_path))
l_cells_data = v_cells_data.GetLayer()
x_res = 25000
y_res = 25000
x_min, x_max, y_min, y_max = l_cells_data.GetExtent()

no_data_value = -1

count = 0
list_files = []
for i, attr in enumerate(sorted_attr_dict):
    #if i < 10:
    #    continue
    count +=1
    target_ds = gdal.Rasterize(
        str(tiff_path / f'{attr}.tif'),
        str(cells_path),
        xRes = x_res,
        yRes = y_res,
        attribute  = attr,
        outputBounds=[x_min, y_min, x_max, y_max],
        noData = no_data_value,
        outputType=gdal.GDT_Float32
    )
    list_files.append(str(tiff_path / f'{attr}.tif'))
    target_ds = None
    #print(f'{count:02d} - {sorted_attr_dict[attr]}')

output_path.unlink(missing_ok=True)
parameters = ['', '-separate', '-a_nodata', f'{no_data_value}', '-o', str(output_path)] + list_files
gdal_merge.main(parameters)
for f in list_files:
    fp = Path(f)
    fp.unlink()

In [4]:
cells_path = shp_path / 'reference-resunet.shp'
output_path = tiff_path / 'reference-resunet.tif'
col = fiona.open(cells_path)
attr_list = [att for att in col.schema['properties'].keys() if str(att).startswith('b')]
sorted_attr_dict = sorted(attr_list)

v_cells_data = ogr.Open(str(cells_path))
l_cells_data = v_cells_data.GetLayer()
x_res = 25000
y_res = 25000
x_min, x_max, y_min, y_max = l_cells_data.GetExtent()

no_data_value = -1

count = 0
list_files = []
for i, attr in enumerate(sorted_attr_dict):
    #if i < 10:
    #    continue
    count +=1
    target_ds = gdal.Rasterize(
        str(tiff_path / f'{attr}.tif'),
        str(cells_path),
        xRes = x_res,
        yRes = y_res,
        attribute  = attr,
        outputBounds=[x_min, y_min, x_max, y_max],
        noData = no_data_value,
        outputType=gdal.GDT_Float32
    )
    list_files.append(str(tiff_path / f'{attr}.tif'))
    target_ds = None
    #print(f'{count:02d} - {sorted_attr_dict[attr]}')

output_path.unlink(missing_ok=True)
parameters = ['', '-separate', '-a_nodata', f'{no_data_value}', '-o', str(output_path)] + list_files
gdal_merge.main(parameters)
for f in list_files:
    fp = Path(f)
    fp.unlink()